In [1]:
## Career Agent Experiments

import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

## Support for dataset retrieval with Huggingface
from datasets import load_dataset

import cassio

/Users/sukhdeepnarulasingh/AI-ML/Projects/Career-Agent/venv-career/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PyPDF2 import PdfReader
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
pdfreader = PdfReader("source/linkedin.pdf")

In [5]:
from typing_extensions import Concatenate

raw_text = ""
for i, page in enumerate(pdfreader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
    

In [6]:
raw_text.__len__()

4811

In [7]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

In [8]:
ASTRA_DB_ID

'82022842-6e25-40c9-85cf-1378a4a7f65a'

In [9]:
import cassio
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [10]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
## Now create langchain vectorstore

astra_vector_store = Cassandra(
    embedding= embeddings,
    table_name="experiments",
    session= None,
    keyspace=None
)


/var/folders/_t/v38w57x92v12tfgjb78y_mlh0000gn/T/ipykernel_60498/4223115160.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)
/var/folders/_t/v38w57x92v12tfgjb78y_mlh0000gn/T/ipykernel_60498/4223115160.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", "?", "!"],
    chunk_size=1_000,        # characters
    chunk_overlap=200,       # characters overlap
)
texts = splitter.split_text(raw_text)
texts

["Contact\nsukhdeepnarulasingh@gmail.c\nom\nwww.linkedin.com/in/\nsukhdeepsingh1612  (LinkedIn)\nTop Skills\nSwift (Programming Language)\nJava Swing\nSoftware Design\nCertifications\nInitiating and Planning Projects Sukhdeep Singh\nCS Grad @ UBC | Actively looking for Software Engineering Roles\nVancouver, British Columbia, Canada\nSummary\n\u200d Recent CS Graduate | Passionate about Tech & Innovation | Open\nto Exciting Opportunities\nHello, and welcome to my LinkedIn profile! I'm a recent Computer\nScience graduate from the University of British Columbia, driven by a\nrelentless curiosity and a passion for technology. \nThroughout my academic journey, I've had the opportunity to dive\ndeep into the world of algorithms, data structures, and software\ndevelopment. My coursework, projects, and internships have\nequipped me with a strong foundation in computer science and the\nskills required to tackle complex technical challenges.\nAs I embark on the exciting journey of transitioning 

In [12]:
astra_vector_store.add_texts(texts)

print("Inserted %i headlines." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 6 headlines.


In [13]:
retriever = astra_vector_store.as_retriever()


[Document(id='2641b34a50374bc7b385d7142782747c', metadata={}, page_content="Kelowna, British Columbia, Canada\n●Achieved consistent on-time order fulfilment, exceeding performance targets\nby accurately picking and packing items to ensure timely customer delivery.\n●Collaborated effectively with a diverse team to streamline fulfilment\nprocesses, resulting in improved efficiency and a reduction in errors.\n●Prioritized safety protocols and actively contributed to a secure working\nenvironment while also collecting and utilizing customer feedback to enhance\nthe overall shopping experience.\nEducation\nThe University of British Columbia\nBachelor of Science - BS,\xa0Computer Science \xa0·\xa0(September 2019\xa0-\xa0April 2023)\nTrent University\nBachelor's of Computer Science,\xa0Computer Science \xa0·\xa0(2018\xa0-\xa02019)\nVivek High School\nHigh School Diploma,\xa0Science Stream \xa0·\xa0(2016\xa0-\xa02018)\nVivek High School\nHigh School Diploma\xa0 \xa0·\xa0(2003\xa0-\xa02018)\n\x